 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import pandas as pd
import sys
sys.path.append('..')
from lib import Node
from lib import Tree

In [2]:
df = pd.read_pickle('../data/parsed/pickled_data_test.pickle')

In [3]:
### Ju Yun's test suite 11-11

test = df.loc[df['Scenario'] == '401']
test_labeled = test.loc[test['Label'] != '-1']
test = test_labeled[0:100]
#test = test.sample(frac=1)
test = test.reset_index(drop=True)

# print(test.loc[test['From'].apply(lambda x: x[0]) == 'david.forster@enron.com'].index.values)
# print(test.loc[test['From'])
ls = ['From']
#data, depth, benchmark, rows, features, cat_features)
tree = Tree(test, 4, None, range(test.shape[0]), ls, ls)
tree.fit()
print(tree)



Rows: range(0, 100)


split_cat: splitting on address: david.forster@enron.com
split_cat: num left: 3
split_cat: num right: 97


best_breakpoint_this_feature: david.forster@enron.com
No options left
UNR ['registrar.isc@enron.com' 'monika.causholli@enron.com'
 'emily.sellers@enron.com' 'steve.hotte@enron.com' 'rita.houston@enron.com'
 'grs4ferc@starpower.net' 'debbie.doyle@enron.com' 'tim.cooper@enron.com'
 'shelley.corman@enron.com' 'wkaseme@enron.com' 'martin.cuilla@enron.com'
 'james.steffes@enron.com' 'amosher@appanet.org' 'jeff.dasovich@enron.com'
 'ann.schmidt@enron.com' 'bcherry@sempra.com' 'jennifer.rudolph@enron.com'
 'susan.mara@enron.com' 'jermeier@earthlink.net'
 'rebecca.cantrell@enron.com' 'sgovenar@govadv.com' 'kathy.franz@enron.com'
 'hcameron@uclink.berkeley.edu' 'rcarroll@bracepatt.com'
 'jkelly@fairisaac.com' 'rob.bradley@enron.com' 'cameron@perfect.com'
 'sharonda.stephens@enron.com' 'sarah.novosel@enron.com'
 'douglass@arterhadden.com' 'wayne.gardner@enron.com'
 'ow

In [ ]:
rows = [1, 30, 32, 43, 46]

In [ ]:
members = df.loc[rows]['Label'].values

In [ ]:
members

In [ ]:
members == '-1'

In [ ]:
a = members[members == '-1']
a

TODO:

- Dates to some kind of number
- from, to: split using membership
- dates: split on number
- subject: find subjects that are equal, or lsa matrix


In [ ]:
one = df.loc[df['Scenario'] == '401']
one_labeled = one.loc[one['Label'] != '-1']
one_labeled.shape

In [ ]:
def get_first(l):
    return l[0]

In [ ]:
df['From'].apply(lambda x: x[0]).unique()[0]

In [ ]:
##test.loc[test['From'].apply(lambda x: x[0]) == 'david.forster@enron.com'].index.valuestest[0:1]

In [ ]:
test = df.loc[df['Scenario'] == '401']
test_labeled = test.loc[test['Label'] != '-1']
test = test_labeled[0:100]

In [ ]:
test.shape

In [ ]:
# test = test.sample(frac=1)
test = test.reset_index(drop=True)

In [ ]:
ls = ['From']

In [ ]:
rows = [80, 81, 82, 85, 86, 87, 88]
address = 'registrar.isc@enron.com'
feature = 'From'
# uniques = test[feature].apply(lambda x: x[0]).unique()
# to_parse = [(test['From'][x],test['Label'][x]) for x in rows]
# to_parse = pd.DataFrame(to_parse, columns=('From','Label'), index=rows)
uniques = test.loc[rows]['From'].apply(lambda x: x[0]).unique()
# to_parse.loc[to_parse['From'].apply(lambda x: x[0]) == address].index.values
uniques

In [ ]:
#data, depth, benchmark, rows, features, cat_features)
tree = Tree(test, 4, None, range(test.shape[0]), ls, ls)

In [ ]:
tree.fit()
print(tree)

In [ ]:
tree.visualize()

In [ ]:
%%bash
cd vis
dot -Tpng tree.dot -o tree.png
dot -Tsvg tree.dot -o tree.svg

In [ ]:
from IPython.display import Image
Image(filename='vis/tree.png')

In [ ]:
test.loc[test['From'].apply(lambda x: x[0]) == 'david.forster@enron.com'].index.values

In [ ]:
members = test.loc[range(0,100)]
left_members = members.loc[members['From'] == 'david.forster@enron.com'].index.values
members.loc[members['From'].apply(lambda x: x [0] == 'david.forster@enron.com')]